In [7]:
import numpy as np
import networkx as nx
from itertools import combinations
import random
import math  # Import math module

def generate_block_network(block_sizes, prob_matrix):
    """Generate a stochastic block model (SBM) network."""
    num_blocks = len(block_sizes)
    num_nodes = sum(block_sizes)
    G = nx.Graph()
    
    # Assign blocks
    node_labels = []
    current_label = 0
    for i, size in enumerate(block_sizes):
        for _ in range(size):
            G.add_node(current_label, block=i)
            node_labels.append(current_label)
            current_label += 1
    
    # Add edges based on probability matrix
    start_idx = 0
    for i in range(num_blocks):
        end_idx = start_idx + block_sizes[i]
        start_jdx = 0
        for j in range(num_blocks):
            end_jdx = start_jdx + block_sizes[j]
            
            for u in range(start_idx, end_idx):
                for v in range(start_jdx, end_jdx):
                    if u < v and np.random.rand() < prob_matrix[i][j]:
                        G.add_edge(u, v)
            
            start_jdx = end_jdx
        start_idx = end_idx
    
    return G

def modify_network(G, modification_fraction):
    """Randomly remove and add links to create the observed network A_O."""
    G_modified = G.copy()
    edges = list(G.edges())
    nodes = list(G.nodes())
    num_to_modify = int(len(edges) * modification_fraction)
    
    removed_edges = []
    added_edges = []
    
    # Remove edges
    edges_to_remove = np.random.choice(len(edges), num_to_modify // 2, replace=False)
    for idx in edges_to_remove:
        removed_edges.append(edges[idx])
        G_modified.remove_edge(*edges[idx])
    
    # Add edges
    for _ in range(num_to_modify // 2):
        u, v = np.random.choice(nodes, 2, replace=False)
        if not G_modified.has_edge(u, v):
            G_modified.add_edge(u, v)
            added_edges.append((u, v))
    
    return G_modified, removed_edges, added_edges

def initialize_partition(G):
    """Initialize random partitions (each node is its own group)."""
    return {node: node for node in G.nodes()}  # Every node is its own group initially

def calculate_H(P, G):
    """Compute H(P) based on partition P."""
    block_counts = {}
    for node, group in P.items():
        if group not in block_counts:
            block_counts[group] = []
        block_counts[group].append(node)

    lO = {}  # Observed links between blocks
    r = {}   # Possible links between blocks
    
    # Count links in the observed network
    for u, v in G.edges():
        gu, gv = P[u], P[v]
        key = tuple(sorted([gu, gv]))
        lO[key] = lO.get(key, 0) + 1
    
    # Compute rαβ (max possible links between groups)
    for (g1, nodes1), (g2, nodes2) in combinations(block_counts.items(), 2):
        key = tuple(sorted([g1, g2]))
        r[key] = len(nodes1) * len(nodes2)
    
    # Compute H(P)
    H = sum(math.log(float(r[key] + 1)) + math.log(math.comb(r[key], lO.get(key, 0)))
            for key in r)
    
    return H

def metropolis_MCMC(G, num_samples=5):
    """Run MCMC sampling to estimate the partition distribution."""
    P = initialize_partition(G)  # Initial partition
    partitions = []
    
    for _ in range(num_samples):
        node = random.choice(list(G.nodes()))
        new_group = random.choice(list(P.values()))  # Move node to a random group
        
        # Compute ΔH
        old_H = calculate_H(P, G)
        P[node] = new_group
        new_H = calculate_H(P, G)
        delta_H = new_H - old_H
        
        # Accept move with probability exp(-ΔH)
        if delta_H > 0 and np.random.rand() >= np.exp(-delta_H):
            P[node] = node  # Reject the move
        
        partitions.append(P.copy())
    
    return partitions

def estimate_link_reliability(G, partitions):
    """Compute R_L(i,j) for each edge based on sampled partitions."""
    reliability = {edge: 0 for edge in G.edges()}
    
    for P in partitions:
        for u, v in G.edges():
            gu, gv = P[u], P[v]
            key = tuple(sorted([gu, gv]))
            lO = sum(1 for p in partitions if p[u] == p[v])  # Count partitions where u,v are together
            r = len(partitions)
            reliability[(u, v)] += (lO + 1) / (r + 2)
    
    # Normalize
    for edge in reliability:
        reliability[edge] /= len(partitions)
    
    return reliability

# Generate True Network (A_T)
block_sizes = [10, 10, 10]  # Three groups of 10 nodes each
prob_matrix = [[0.8, 0.2, 0.1], [0.2, 0.7, 0.15], [0.1, 0.15, 0.9]]
true_network = generate_block_network(block_sizes, prob_matrix)

# Create Observed Network (A_O)
observed_network, removed, added = modify_network(true_network, modification_fraction=0.2)

# Run MCMC Sampling
partitions = metropolis_MCMC(observed_network, num_samples=10000)

# Estimate Link Reliability
reliability = estimate_link_reliability(observed_network, partitions)

# Print Results
print("Top 10 Most Reliable Links:")
for edge, rl in sorted(reliability.items(), key=lambda x: x[1], reverse=True)[:10]:
    print(f"Edge {edge}: Reliability {rl:.4f}")


KeyboardInterrupt: 